In [1]:
import pandas as pd

data = pd.read_csv("../../data/phemernr2-tf_dataset.csv", lineterminator="\n")
print(data.shape)
data.head()

(1705, 5)


,tweet_id,tweet_text,label,topic,tvt2
0,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,True,germanwings-crash-all-rnr-threads,training
1,552798891994009601,Reports that two of the dead in the #CharlieHe...,True,charliehebdo-all-rnr-threads,testting
2,580352540316946432,'No survivors' in #Germanwings crash says Fren...,False,germanwings-crash-all-rnr-threads,training
3,524983403775799297,Tragedy mounts as soldier shot this AM dies of...,True,ottawashooting-all-rnr-threads,training
4,544511915158810624,Watch the moment gunfire and explosions were h...,True,sydneysiege-all-rnr-threads,training


In [2]:
length = data.shape[0]
length

1705

In [3]:
import random

# random.seed(33)

train_w = [0 for i in range(675)]
val_w = [1 for i in range(225)]
test_w = [2 for i in range(100)]

weights = train_w + val_w + test_w
print(f"weights : {len(weights)}")

tvt = []
for i in range(length):
    gacha = random.sample(weights, 1)[0]
    if gacha == 0:
        tvt.append("training")
    elif gacha == 1:
        tvt.append("validation")
    else:
        tvt.append("testting")

print(f"Training : {tvt.count('training')} - {round(tvt.count('training')/length, 3)}")
print(f"Validation : {tvt.count('validation')} - {round(tvt.count('validation')/length, 3)}")
print(f"Testing : {tvt.count('testting')} - {round(tvt.count('testting')/length, 3)}")

weights : 1000
Training : 1176 - 0.69
Validation : 371 - 0.218
Testing : 158 - 0.093


In [4]:
data['tvt2'] = pd.Series(tvt)
data.head()

,tweet_id,tweet_text,label,topic,tvt2
0,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,True,germanwings-crash-all-rnr-threads,training
1,552798891994009601,Reports that two of the dead in the #CharlieHe...,True,charliehebdo-all-rnr-threads,training
2,580352540316946432,'No survivors' in #Germanwings crash says Fren...,False,germanwings-crash-all-rnr-threads,training
3,524983403775799297,Tragedy mounts as soldier shot this AM dies of...,True,ottawashooting-all-rnr-threads,training
4,544511915158810624,Watch the moment gunfire and explosions were h...,True,sydneysiege-all-rnr-threads,training


In [5]:
def bool2str(df):
    if df['label'] == True:
        return "true"
    else:
        return "false"

data['label'] = data.apply(lambda row: bool2str(row), axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1705 entries, 0 to 1704
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_id    1705 non-null   int64 
 1   tweet_text  1705 non-null   object
 2   label       1705 non-null   object
 3   topic       1705 non-null   object
 4   tvt2        1705 non-null   object
dtypes: int64(1), object(4)
memory usage: 66.7+ KB


In [6]:
label_count = data['label'].value_counts()
label_count

true     1067
false     638
Name: label, dtype: int64

In [7]:
combination = data.apply(lambda row: f"{row['label']}_{row['tvt2']}", axis=1).value_counts()
comparison = {}
for k, comb in combination.items():
    cv_fold = k.split("_")[1]
    label = k.split("_")[0]

    if cv_fold not in comparison:
        comparison[cv_fold] = {}
    
    comparison[cv_fold][label] = comb

labels = data['label'].unique().tolist()

def label_ratio(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{round(label_dict[l]/total, 2)}\t"
    
    return report[:-1]

def label_raw_value(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{label_dict[l]}\t"
    
    return report[:-1]

labels_str = ','.join([str(l) for l in labels])
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_ratio(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_ratio(comp, labels)}")
    
labels_str = ','.join([str(l) for l in labels])
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_raw_value(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_raw_value(comp, labels)}")


Label,true,false
Original	0.63	0.37
Training	0.63	0.37
Validation	0.61	0.39
Testting	0.61	0.39

Label,true,false
Original	1067	638
Training	742	434
Validation	228	143
Testting	97	61


In [9]:
data.to_csv("../../data/phemernr2-tf_dataset.csv", index=False)